# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Data
0,0,yellowknife,62.46,-114.35,-31.00,66,5,4.70,CA,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."
1,1,berlevag,70.86,29.09,25.11,70,12,13.44,NO,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."
2,2,beloha,-25.17,45.05,95.85,27,0,4.94,MG,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."
3,3,khatanga,71.97,102.50,-2.63,97,100,7.11,RU,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."
4,4,namibe,-15.20,12.15,80.60,69,40,5.30,AO,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#narrowing down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                    & (city_data_df["Wind Speed"] < 10) \
                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Data
137,137,gambissara,13.24,-14.31,72.79,12,0,9.24,SN,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."
255,255,pasni,25.26,63.47,70.32,16,0,9.57,PK,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."
284,284,murray bridge,-35.12,139.27,73.99,62,0,5.99,AU,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."
298,298,champerico,14.30,-91.92,74.79,64,0,4.74,GT,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."
373,373,akyab,20.15,92.90,78.46,46,0,9.53,MM,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."
399,399,sotouboua,8.57,0.98,79.43,18,0,6.06,TG,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."
410,410,ibra,22.69,58.53,70.66,20,0,3.11,OM,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."
414,414,mankono,8.06,-6.19,79.03,23,0,5.57,CI,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."
415,415,bouna,9.27,-3.00,78.89,15,0,7.90,CI,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."
448,448,tezpur,26.63,92.80,74.25,61,0,2.71,IN,"[{'City': 'yellowknife', 'Lat': 62.46, 'Lng': ..."


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#create a dataframe called hotel_df to store hotel name along with city
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
137,gambissara,SN,13.24,-14.31,
255,pasni,PK,25.26,63.47,
284,murray bridge,AU,-35.12,139.27,
298,champerico,GT,14.30,-91.92,
373,akyab,MM,20.15,92.90,
399,sotouboua,TG,8.57,0.98,
410,ibra,OM,22.69,58.53,
414,mankono,CI,8.06,-6.19,
415,bouna,CI,9.27,-3.00,
448,tezpur,IN,26.63,92.80,


In [7]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    #get latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    #use the search term: hotel and out lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except(KeyError, IndexError):
        print("Missing field/result...skipping")
hotel_df

Missing field/result...skipping


,City,Country,Lat,Lng,Hotel Name
137,gambissara,SN,13.24,-14.31,
255,pasni,PK,25.26,63.47,3G hotel pasni
284,murray bridge,AU,-35.12,139.27,Balcony On Sixth Lodge
298,champerico,GT,14.30,-91.92,"Hotel y restaurante ""El Diamante"""
373,akyab,MM,20.15,92.90,Hotel Memory
399,sotouboua,TG,8.57,0.98,Hôtel La Planète
410,ibra,OM,22.69,58.53,مزرعة راحة النفوس المنزفة
414,mankono,CI,8.06,-6.19,Hotel Cajoux
415,bouna,CI,9.27,-3.00,Hotel Reserve
448,tezpur,IN,26.63,92.80,Hotel Kanyapur


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))